## Baseline

Como baseline do projeto utilizei a rede neural simples, que é comumente utilizada no ensino de classificação

http://archive.ics.uci.edu/ml/datasets/Iris

In [1]:
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


### Leitura e conversão dos dados

In [5]:
# load dataset
dataframe = pandas.read_csv("data/train_v2.csv", header=1)
dataframe = dataframe.apply(pandas.to_numeric, errors='coerce')
dataset = dataframe.values
X = dataset[:,1:770].astype(float)
Y = dataset[:,770]

## Geração de variáveis de classificação

A variável de saída, "loss",  pode conter valores inteiros entre 0 e 100;

Uma maneira de se modelar o problema é considerar cada possibilidade (0,1,2...100) como uma classe

Ao modelar problemas de classificação de várias classes usando redes neurais, é uma boa prática remodelar o atributo de saída de um vetor que contém valores para cada valor de classe para ser uma matriz com um booleano para cada valor de classe e se uma determinada instância possui ou não esse valor. valor de classe ou não.

Isso é chamado de codificação quente ou criação de variáveis fictícias a partir de uma variável categórica.

Por exemplo: 
    

In [21]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

dummy_y.shape

(105470, 89)

No caso, foi gerado uma matriz de 105470 linhas com 89 colunas.
Cada uma das 89 colunas corresponde a um dos 100 valores.
Poderiam haver entre 1 e 100 colunas, porém a amostra utilizada, apenas 89 valores foram encontrados.

## Definição do modelo

A biblioteca Keras fornece classes que para permitem o uso de modelos de rede neural desenvolvidos com no scikit-learn.

Há uma classe KerasClassifier que pode ser usada como estimador no scikit-learn, o tipo de modelo básico na biblioteca. 
O KerasClassifier leva o nome de uma função como argumento. 
Esta função irá retornar o modelo de rede neural construído, pronto para o treinamento.

Abaixo está uma função que criará uma rede neural *que servirá de baseline para outros modelos* para o problema de classificação. 
Ele cria uma rede totalmente conectada simples com uma camada oculta que contém 50 neurônios.

A camada oculta usa uma função de ativação do *relu*, que é uma boa prática. 
Como usamos uma codificação one-hot para nosso conjunto de dados, a camada de saída deve criar 89 valores de saída, um para cada classe. 
O valor de saída com o maior valor será considerado como a classe prevista pelo modelo.

A topologia de rede dessa rede neural simples de uma camada pode ser resumida como:

768 etradas -> 50 nós ocultos -> 89 saídas

Observe que usamos uma função de ativação "softmax" na camada de saída. 
Isso foi usado para garantir que os valores de saída estejam na faixa de 0 e 1

Por fim, a rede usa o algoritmo de otimização de descida de gradiente Adam com uma função de perda logarítmica, chamada de "categorical_crossentropy" em Keras.


In [23]:
# define baseline model
def baseline_model():
	model = Sequential()
	model.add(Dense(50, input_dim=769, activation='relu'))
	model.add(Dense(89, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

## Avaliação do modelo com validação cruzada do k-Fold

Agora podemos avaliar o modelo com os dados de treinamento.

Usei o scikit-learn para avaliar o modelos, usando a validação cruzada k-fold.

Primeiro, podemos definir o procedimento de avaliação do modelo. 
Definimos o número 10 épocas e embaralhamos os dados antes de particioná-los.
Defini o batch_size em 500;

In [25]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=10, batch_size=500, verbose=1)
kfold = KFold(n_splits=5, shuffle=True)

Agora podemos avaliar nosso modelo (usado o estimador) no conjunto de dados (X e dummy_y) usando um procedimento de validação cruzada de 10 épocas (kfold).

A avaliação do modelo leva algum tempo, dada a quantidade de dados utilizada para treinar, o número de épocas (10) e a quantidade de vezes que será realizado (5)


In [26]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold)

Epoch 1/10
84376/84376 [==============================] - 5s 54us/step - loss: nan - accuracy: 0.9063
Epoch 2/10
84376/84376 [==============================] - 4s 46us/step - loss: nan - accuracy: 0.9071
Epoch 3/10
84376/84376 [==============================] - 4s 46us/step - loss: nan - accuracy: 0.9071
Epoch 4/10
84376/84376 [==============================] - 4s 46us/step - loss: nan - accuracy: 0.9071
Epoch 5/10
84376/84376 [==============================] - 4s 47us/step - loss: nan - accuracy: 0.9071
Epoch 6/10
84376/84376 [==============================] - 4s 46us/step - loss: nan - accuracy: 0.9071
Epoch 7/10
84376/84376 [==============================] - 4s 46us/step - loss: nan - accuracy: 0.9071
Epoch 8/10
84376/84376 [==============================] - 4s 47us/step - loss: nan - accuracy: 0.9071
Epoch 9/10
84376/84376 [==============================] - 4s 45us/step - loss: nan - accuracy: 0.9071
Epoch 10/10
21094/21094 [==============================] - 1s 34us/step
Epoch 1/10

Finalmente podemos obter a média da acuracidade das 5 execusões, e o desvio padrão entre as 5.

In [27]:
print('-'*100)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

----------------------------------------------------------------------------------------------------
Baseline: 90.72% (0.16%)


Um modelo simples nos permitiu ter a acuracidade de 90.72%. 
Irei utilizar este valor como base de comparação em outras abordagens.

* utilizei como guia o notebook https://www.kaggle.com/pierrek20/multiclass-iris-prediction-with-tensorflow-keras